In [1]:
# Copyright (c) OpenMMLab. All rights reserved.
import logging
import mimetypes
import os
import time
from argparse import ArgumentParser

import cv2
import json_tricks as json
import mmcv
import mmengine
import numpy as np
from mmengine.logging import print_log

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples, split_instances
from mmpose.utils import adapt_mmdet_pipeline

try:
	from mmdet.apis import inference_detector, init_detector
	has_mmdet = True
except (ImportError, ModuleNotFoundError):
	has_mmdet = False


from mmengine.config import Config, ConfigDict
from typing import Union
from mmcv.transforms import Compose
from typing import Optional, Sequence, Union
from mmcv.ops import RoIPool

import torch
from PIL import Image

In [2]:
model_config = r'C:\Users\user\Documents\GitHub\mmpose\temp_modify\custom_config\HMD_mo2cap2_config.py'
model_ckpt = r'C:\Users\user\Documents\GitHub\mmpose\temp_modify\work_dirs\HMD_mo2cap2\epoch_3.pth'
device = 'cuda'

In [3]:
# build pose estimator
pose_estimator = init_pose_estimator(
	model_config,
	model_ckpt,
	device=device,
	# cfg_options=dict(
	# 	model=dict(test_cfg=dict(output_heatmaps=args.draw_heatmap)))
	)
visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)

Loads checkpoint by local backend from path: C:\Users\user\Documents\GitHub\mmpose\temp_modify\work_dirs\HMD_mo2cap2\epoch_3.pth


In [4]:
pose_estimator.cfg

Config (path: C:\Users\user\Documents\GitHub\mmpose\temp_modify\custom_config\HMD_mo2cap2_config.py): {'default_scope': 'mmpose', 'default_hooks': {'checkpoint': {'type': 'CheckpointHook', 'interval': 10, 'max_keep_ckpts': 3}, 'visualization': {'type': 'PoseVisualizationHook', 'enable': True, 'interval': 15, 'kpt_thr': 0.3}}, 'custom_hooks': [{'type': 'SyncBuffersHook'}], 'env_cfg': {'cudnn_benchmark': False, 'mp_cfg': {'mp_start_method': 'fork', 'opencv_num_threads': 0}, 'dist_cfg': {'backend': 'nccl'}}, 'log_processor': {'type': 'LogProcessor', 'window_size': 50, 'by_epoch': True, 'num_digits': 6}, 'log_level': 'INFO', 'load_from': None, 'resume': False, 'backend_args': {'backend': 'local'}, 'train_cfg': {'type': 'EpochBasedTrainLoop', 'max_epochs': 10, 'val_interval': 1, 'dynamic_intervals': [(280, 1)]}, 'val_cfg': {}, 'test_cfg': {}, 'auto_scale_lr': {'base_batch_size': 256}, 'optim_wrapper': {'optimizer': {'type': 'Adam', 'lr': 0.0005}}, 'param_scheduler': [{'type': 'LinearLR', 'b

In [5]:
visualizer.set_dataset_meta(
	pose_estimator.dataset_meta, skeleton_style='mo2cap2')

In [6]:
args = {
	'det_config': 'path/to/det_config',  # detection 설정 파일 경로
	'det_checkpoint': 'path/to/det_checkpoint',  # detection 체크포인트 파일 경로
	'pose_config': 'path/to/pose_config',  # 포즈 설정 파일 경로
	'pose_checkpoint': 'path/to/pose_checkpoint',  # 포즈 체크포인트 파일 경로
	'input': '',  # 이미지/비디오 파일 경로
	'show': False,  # 이미지를 표시할지 여부
	'output_root': '',  # 출력 이미지 파일 루트 경로
	'save_predictions': False,  # 예측 결과를 저장할지 여부
	'device': 'cuda:0',  # 추론에 사용할 장치
	'det_cat_id': 0,  # 바운딩 박스 탐지 모델의 카테고리 ID
	'bbox_thr': 0.3,  # 바운딩 박스 점수 임계값
	'nms_thr': 0.3,  # 바운딩 박스 NMS를 위한 IoU 임계값
	'kpt_thr': 0.3,  # 키포인트 시각화 임계값
	'draw_heatmap': False,  # 모델이 예측한 히트맵을 그릴지 여부
	'show_kpt_idx': False,  # 키포인트의 인덱스를 표시할지 여부
	'skeleton_style': 'mmpose',  # 스켈레톤 스타일 선택
	'radius': 3,  # 시각화할 키포인트 반경
	'thickness': 1,  # 시각화할 링크 두께
	'show_interval': 0,  # 프레임당 대기 시간 (초)
	'alpha': 0.8,  # 바운딩 박스의 투명도
	'draw_bbox': False,  # 인스턴스의 바운딩 박스를 그릴지 여부
	'Visualizer':'CustomPose3dLocalVisualizer',
}

In [7]:


def get_test_pipeline_cfg(cfg: Union[str, ConfigDict]) -> ConfigDict:
    """Get the test dataset pipeline from entire config.

    Args:
        cfg (str or :obj:`ConfigDict`): the entire config. Can be a config
            file or a ``ConfigDict``.

    Returns:
        :obj:`ConfigDict`: the config of test dataset.
    """
    if isinstance(cfg, str):
        cfg = Config.fromfile(cfg)

    def _get_test_pipeline_cfg(dataset_cfg):
        if 'pipeline' in dataset_cfg:
            return dataset_cfg.pipeline
        # handle dataset wrapper
        elif 'dataset' in dataset_cfg:
            return _get_test_pipeline_cfg(dataset_cfg.dataset)
        # handle dataset wrappers like ConcatDataset
        elif 'datasets' in dataset_cfg:
            return _get_test_pipeline_cfg(dataset_cfg.datasets[0])

        raise RuntimeError('Cannot find `pipeline` in `test_dataloader`')

    return _get_test_pipeline_cfg(cfg.test_dataloader.dataset)

get_test_pipeline_cfg(pose_estimator.cfg)

[{'type': 'LoadImage'},
 {'type': 'GetBBoxCenterScale', 'padding': 1.0},
 {'type': 'TopdownAffine', 'input_size': (256, 256)},
 {'type': 'PackPoseInputs'}]

In [8]:
ImagesType = Union[str, np.ndarray, Sequence[str], Sequence[np.ndarray]]

def inference_detector(
    # model: nn.Module,
    imgs: ImagesType,
    test_pipeline: Optional[Compose] = None,
    text_prompt: Optional[str] = None,
    custom_entities: bool = False,
):
    """Inference image(s) with the detector.

    Args:
        model (nn.Module): The loaded detector.
        imgs (str, ndarray, Sequence[str/ndarray]):
           Either image files or loaded images.
        test_pipeline (:obj:`Compose`): Test pipeline.

    Returns:
        :obj:`DetDataSample` or list[:obj:`DetDataSample`]:
        If imgs is a list or tuple, the same length list type results
        will be returned, otherwise return the detection results directly.
    """

    if isinstance(imgs, (list, tuple)):
        is_batch = True
    else:
        imgs = [imgs]
        is_batch = False

    # cfg = model.cfg

    if test_pipeline is None:
        # cfg = cfg.copy()
        test_pipeline = get_test_pipeline_cfg(pose_estimator.cfg)
        if isinstance(imgs[0], np.ndarray):
            # Calling this method across libraries will result
            # in module unregistered error if not prefixed with mmdet.
            test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'

        test_pipeline = Compose(test_pipeline)

    # if model.data_preprocessor.device.type == 'cpu':
    #     for m in model.modules():
    #         assert not isinstance(
    #             m, RoIPool
    #         ), 'CPU inference with RoIPool is not supported currently.'

    result_list = []
    for i, img in enumerate(imgs):
        # prepare data
        if isinstance(img, np.ndarray):
            # TODO: remove img_id.
            data_ = dict(img=img, img_id=0)
        else:
            # TODO: remove img_id.
            data_ = dict(img_path=img, img_id=0)

        if text_prompt:
            data_['text'] = text_prompt
            data_['custom_entities'] = custom_entities

        # build the data pipeline
        data_ = test_pipeline(data_)

        data_['inputs'] = [data_['inputs']]
        data_['data_samples'] = [data_['data_samples']]

        # forward the model
        # with torch.no_grad():
        #     results = model.test_step(data_)[0]

        # result_list.append(results)

    if not is_batch:
        return result_list[0]
    else:
        return result_list

In [9]:

def process_one_image(
					args,
					img,
					# detector,
					pose_estimator,
					visualizer=None,
					show_interval=0
					):
	"""Visualize predicted keypoints (and heatmaps) of one image."""

	# predict bbox
	# det_result = inference_detector(img)
	# pred_instance = det_result.pred_instances.cpu().numpy()



	# img = Image.open(img)
	# w,h = img.size

	# bboxes = [0,0,w,h]

	# bboxes = np.concatenate(
	# 	(pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
	# bboxes = bboxes[np.logical_and(pred_instance.labels == args['det_cat_id'],
	# 							   pred_instance.scores > args['bbox_thr'])]
	# bboxes = bboxes[nms(bboxes, args['nms_thr']), :4]
	'''
	bbox_format (str): The bbox format indicator. Options are ``'xywh'``
		and ``'xyxy'``. Defaults to ``'xyxy'``
	'''
	# predict keypoints
	pose_results = inference_topdown(pose_estimator, img)
	data_samples = merge_data_samples(pose_results)

	# show the results
	if isinstance(img, str):
		img = mmcv.imread(img, channel_order='rgb')
	elif isinstance(img, np.ndarray):
		img = mmcv.bgr2rgb(img)

	if visualizer is not None:
		visualizer.add_datasample(
			'result',
			image = img,
			data_sample = data_samples,
			draw_gt = False,
			draw_pred = True,
			draw_2d = True,
			draw_bbox = False,
			show_kpt_idx = False,
			skeleton_style = 'mmpose',
			dataset_2d = 'coco',
			dataset_3d = 'coco',
			convert_keypoint = False,
			axis_azimuth = 80,
			axis_limit = 1.7,
			axis_dist = 10.0,
			axis_elev = -170.0,
			num_instances = -1,
			show = True,
			wait_time = 0,
			out_file = None,
			kpt_thr = 0.3,
			step = 0) 

	# if there is no instance detected, return None
	return data_samples.get('pred_instances', None)


In [10]:
raise

RuntimeError: No active exception to reraise

In [12]:
test_img_path = r'F:\mo2cap2_data_temp_extracted\TestSet\weipeng_studio\rgba\frame_c_0_f_0387.png'
train_img_path = r'F:\mo2cap2_data_temp_extracted\TrainSet\mo2cap2_chunk_0002\rgba\mo2cap2_chunk_0002_000000.png'
# inference
pred_instances = process_one_image(args,train_img_path,pose_estimator, visualizer)
pred_instances

170.0 80


<InstanceData(

    META INFORMATION

    DATA FIELDS
    bbox_scores: array([1.], dtype=float32)
    keypoints: array([[[133., 199.],
                [167., 185.],
                [165., 159.],
                [165., 141.],
                [ 99., 179.],
                [107., 157.],
                [111., 141.],
                [145., 153.],
                [141., 143.],
                [141., 143.],
                [143., 135.],
                [127., 153.],
                [131., 143.],
                [131., 143.],
                [129., 135.]]], dtype=float32)
    keypoint_scores: array([[0.9150333 , 0.8262009 , 0.7571056 , 0.78994286, 0.84088075,
                0.8486061 , 0.7861284 , 0.8857001 , 0.65115947, 0.63794756,
                0.5699916 , 0.9174322 , 0.7005327 , 0.68346334, 0.6202189 ]],
              dtype=float32)
    keypoints_visible: array([[0.9150333 , 0.8262009 , 0.7571056 , 0.78994286, 0.84088075,
                0.8486061 , 0.7861284 , 0.8857001 , 0.65115947, 0

In [ ]:
raise

RuntimeError: No active exception to reraise

In [ ]:


if args.save_predictions:
	pred_instances_list = split_instances(pred_instances)

if output_file:
	img_vis = visualizer.get_image()
	mmcv.imwrite(mmcv.rgb2bgr(img_vis), output_file)